In [2]:
import pandas as pd
import plotly.graph_objects as go
import plotly.tools as tls
from plotly.subplots import make_subplots
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
master_df = pd.read_csv('ieso_ga_master_dataset_allWeather_updated2020.csv')
forecasts = pd.read_csv('ga_forecasts_top_12.csv')

In [4]:
master_df.head()

,adjusted_demand_MW,demand_MW,hour_ending_eastern,season,timestamp_eastern,timestamp_utc,altitude_Toronto,apparentTemperature_Toronto,azimuth_Toronto,cloudCover_Toronto,...,precipType_Milton,pressure_Milton,summary_Milton,temperature_Milton,uvIndex_Milton,visibility_Milton,windBearing_Milton,windGust_Milton,windSpeed_Milton,population_Milton
0,11972.0,12103.0,1,2013-2014,2013-05-01 01:00:00,2013-05-01 06:00:00,0.0,13.15,0.0,0.98,...,NaN,1022.1,Mostly Cloudy,12.47,0.0,16.093,165.0,2.60,1.65,101715.0
1,11654.0,11814.0,2,2013-2014,2013-05-01 02:00:00,2013-05-01 07:00:00,0.0,14.02,0.0,0.98,...,NaN,1022.5,Mostly Cloudy,12.36,0.0,16.093,160.0,1.86,1.56,101715.0
2,11492.0,11662.0,3,2013-2014,2013-05-01 03:00:00,2013-05-01 08:00:00,0.0,12.99,0.0,0.98,...,NaN,1022.9,Mostly Cloudy,11.72,0.0,16.093,161.0,2.07,1.57,101715.0
3,11518.0,11675.0,4,2013-2014,2013-05-01 04:00:00,2013-05-01 09:00:00,0.0,13.41,0.0,0.97,...,NaN,1023.3,Mostly Cloudy,11.05,0.0,16.093,140.0,1.57,1.35,101715.0
4,12066.0,12298.0,5,2013-2014,2013-05-01 05:00:00,2013-05-01 10:00:00,0.0,12.70,0.0,0.98,...,NaN,1024.2,Overcast,11.03,0.0,16.093,127.0,1.92,1.19,101715.0


In [5]:
forecasts.head(24)

,forecast,ts,ts_future,horizon_hr
0,0.0000,2016-05-08 01:00:00,2016-05-08 02:00:00,1.0
1,0.0000,2016-05-08 01:00:00,2016-05-08 03:00:00,2.0
2,0.0000,2016-05-08 01:00:00,2016-05-08 04:00:00,3.0
3,0.0000,2016-05-08 01:00:00,2016-05-08 05:00:00,4.0
4,0.0000,2016-05-08 01:00:00,2016-05-08 06:00:00,5.0
5,0.0000,2016-05-08 01:00:00,2016-05-08 07:00:00,6.0
6,0.0000,2016-05-08 01:00:00,2016-05-08 08:00:00,7.0
7,0.0575,2016-05-08 01:00:00,2016-05-08 09:00:00,8.0
8,0.0200,2016-05-08 01:00:00,2016-05-08 10:00:00,9.0
9,0.0000,2016-05-08 01:00:00,2016-05-08 11:00:00,10.0


In [6]:
master_df.rename(columns={'timestamp_eastern': 'ts'}, inplace=True)
df = master_df[['demand_MW', 'ts']].merge(forecasts, on='ts')

In [7]:
df['ts'] = pd.to_datetime(df['ts'])
df['ts_future'] = pd.to_datetime(df['ts_future'])
cols_to_keep = ['ts', 'ts_future', 'demand_MW', 'forecast', 'horizon_hr']
df = df[cols_to_keep]

In [8]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)

fig.append_trace({'x': df.ts, 'y': df.demand_MW, 'type': 'scatter', 'name': 'GA Load'}, 1, 1)
df_future = df[df.ts == df.ts.min()]
fig.append_trace({'x': df_future.ts_future, 'y': df_future.forecast, 'type': 'scatter', 'name': 'forecast', 'line_shape' : 'hv'}, 2, 1)

f = go.FigureWidget(fig)

scatter1 = f.data[0]
scatter2 = f.data[1]
f.layout.hovermode = 'closest'

# create our callback function
def update_point(trace, points, selector):
    try:
        xs = points.xs[0]
    except Exception:
        xs = points.xs
    
    df_future = df[df.ts == xs]
    with f.batch_update():
        scatter2.x = df_future.ts_future
        scatter2.y = df_future.forecast

scatter1.on_click(update_point)
f.update_layout(
        height=800,
)

f

FigureWidget({
    'data': [{'name': 'GA Load',
              'type': 'scatter',
              'uid': '873e8a4…